In [1]:
import pandas as pd

# Read files

In [ ]:
df=pd.read_csv("xyz.csv",header=None)
df=pd.read_excel("xyz.xlsx", sheet_name=0, header=0)
df=pd.read_json("xyz.json", lines=True)

# Basic functions

In [ ]:
df.head(5)     # display the first 5 rows (default is 10)
df.tail(5)     # display the last 5 rows (default is 10)
df.transpose() # transpose of dataframe

df.shape     # dimensions of the dataset i.e. (n_rows,n_columns)
df.shape[0]  # number of rows
df.shape[1]  # number of columns

df.info()     # schema of dataset (columns and their datatypes)
df.describe() # statistics of each column i.e. min, max, quartiles, etc.

df.isnull().sum()  # number of missing values in each column
df.dropna()        # drop rows with missing values
df.fillna(0, inplace=True) # fill missing values with 0, inplace will not create new DF, it will modify current

df.drop_duplicates(inplace=True)  # drop duplicate rows

# loc, iloc

In [ ]:
df.iloc[:3]        # extract first 3 rows i.e. rows 0, 1, 2
df.iloc[1:3, 0:3]  # extract  rows 1 and 2, columns 0-2 

df.loc[df['age']>18,'eligibility']=True  # in the dataset where age>18, set eligibility=True

# assign

In [2]:
df = pd.DataFrame({'temp_c': [17.0, 25.0]}, index=['Portland', 'Berkeley'])
print(df)
df = df.assign(temp_c=df['temp_c'] +1)
print(df)
df = df.assign(temp_f=df['temp_c'] * 9 / 5 + 32)
print(df)

          temp_c
Portland    17.0
Berkeley    25.0
          temp_c
Portland    18.0
Berkeley    26.0
          temp_c  temp_f
Portland    18.0    64.4
Berkeley    26.0    78.8


# apply

In [3]:
import numpy as np
df=pd.DataFrame([[4,25],[9,16]],columns=['A','B'])
df.apply(lambda x: np.sqrt(x))          # apply the lambda function to each row all columns
df['A'].apply(lambda x: np.sqrt(x))     # apply the lambda function to each row, A column only

0    2.0
1    3.0
Name: A, dtype: float64

# SQL equivalence

In [ ]:
#SQL syntax: from[select][where].groupby().agg().rename().reset_index().sort_values()
df[['userId','page']][df['page'=='Thumbs_up']].groupby('userId').count().rename(columns={'page':'total_thumbs_up'}).reset_index()

# where clause can have ==, >, <, >=, <=, between, and, or
df[df['created_at'].between('2020-01-01', '2020-01-31')]

# Aggregate functions -> count(), sum(), mean(), size(), std(), var()
# If we want to use multiple aggregate functions together
df.agg(['sum','min'])

In [ ]:
df.drop_duplicates().replace( ['F','M'], ['0','1'], 'gender'] ) # in gender column replace F by 0, M by 1

# Joins

In [ ]:
pd.merge(df1, df2, how=‘inner’, on=’key’, left_on=’left_key’, right_on=’right_key’)
#how -> inner (default), left, right, outer, cross
#on -> common key/column in both tables
#left_on -> if no common key, column to use from left table 
#right_on -> if no common key, column to use from right table

# Pivot table

In [ ]:
df2=df1.pivot_table(index=['A'],columns=['B'],values='x').reset_index()

# Example

In [4]:
df=pd.DataFrame({"id":["s1,d1","s1,d1","s1,d2","s1,d2","s2,d1","s2,d1","s2,d2","s2,d2"],"date":["2020-01-01","2020-01-03","2020-01-01","2020-01-03","2020-01-01","2020-01-02","2020-01-01","2020-01-02"],"cost":[30,100,70,100,90,150,210,250]})
df

,id,date,cost
0,"s1,d1",2020-01-01,30
1,"s1,d1",2020-01-03,100
2,"s1,d2",2020-01-01,70
3,"s1,d2",2020-01-03,100
4,"s2,d1",2020-01-01,90
5,"s2,d1",2020-01-02,150
6,"s2,d2",2020-01-01,210
7,"s2,d2",2020-01-02,250


In [5]:
# Keep only the first string of id column
df['id']=df['id'].apply(lambda x: x.split(","))
df['id']=df['id'].apply(lambda x: x[0])
df

,id,date,cost
0,s1,2020-01-01,30
1,s1,2020-01-03,100
2,s1,2020-01-01,70
3,s1,2020-01-03,100
4,s2,2020-01-01,90
5,s2,2020-01-02,150
6,s2,2020-01-01,210
7,s2,2020-01-02,250


In [6]:
# find sum for each id for each date
df2=df.groupby(['id','date'])['cost'].sum().reset_index(name = 'sum')
print(df2)

   id        date  sum
0  s1  2020-01-01  100
1  s1  2020-01-03  200
2  s2  2020-01-01  300
3  s2  2020-01-02  400


In [7]:
# for each id all available dates should be shown
df3=df2.pivot_table("sum", "date", "id", fill_value=0).unstack().reset_index(name='sum')
print(df3)

   id        date  sum
0  s1  2020-01-01  100
1  s1  2020-01-02    0
2  s1  2020-01-03  200
3  s2  2020-01-01  300
4  s2  2020-01-02  400
5  s2  2020-01-03    0


In [8]:
# convert dates as well as sum for each date into list form
df4=df3.groupby('id').agg(list).reset_index()
print(df4)

   id                                  date            sum
0  s1  [2020-01-01, 2020-01-02, 2020-01-03]  [100, 0, 200]
1  s2  [2020-01-01, 2020-01-02, 2020-01-03]  [300, 400, 0]
